In [1]:
from openbb import obb
import pandas as pd
import numpy as np
import time

In [2]:
def _fetch_ohlc_openbb(symbol: str, start_date: str = "2010-01-01", end_date: str = None,
                       provider: str = "yfinance", max_retries: int = 3, retry_delay: int = 5) -> pd.DataFrame:
    """
    从 OpenBB 获取标的历史日线 OHLC 数据，转为统一格式 DataFrame。
    列名统一为: Open, High, Low, Close, Adj Close（无 adj_close 时用 close）。
    失败时自动重试 max_retries 次，每次间隔 retry_delay 秒。
    """
    last_err = None
    for attempt in range(max_retries):
        try:
            result = obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="1d",
                provider=provider,
            )
            df = result.to_df()
            break
        except Exception as e:
            last_err = e
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise last_err
    if df is None or df.empty:
        raise ValueError(f"OpenBB 未返回数据: {symbol}")
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)
    col_map = {"open": "Open", "high": "High", "low": "Low", "close": "Close"}
    df = df.rename(columns=col_map)
    if "adj_close" in df.columns:
        df["Adj Close"] = df["adj_close"]
    else:
        df["Adj Close"] = df["Close"]
    df = df[["Open", "High", "Low", "Close", "Adj Close"]].copy()
    df = df.sort_index(ascending=False)
    return df

In [3]:
RESAMPLE_DAYS = {"daily": 1, "weekly": 5, "monthly": 20, "3month": 60}


def _resample_ohlc(df_daily: pd.DataFrame, n_days: int) -> pd.DataFrame:
    """
    将日线 OHLC 按 n_days 聚合成周期线。df 需按时间降序（最新在前）。
    Open=周期首日, High=max, Low=min, Close=周期末日, Adj Close=周期末日。
    """
    ohlc = df_daily[["Open", "High", "Low", "Close", "Adj Close"]].copy()
    if isinstance(ohlc.columns, pd.MultiIndex):
        ohlc.columns = ohlc.columns.get_level_values(-1)
    rows = []
    for i in range(0, len(ohlc), n_days):
        chunk = ohlc.iloc[i : i + n_days]
        if len(chunk) < n_days:
            continue
        rows.append({
            "Open": float(chunk["Open"].iloc[-1]),
            "High": float(chunk["High"].max()),
            "Low": float(chunk["Low"].min()),
            "Close": float(chunk["Close"].iloc[0]),
            "Adj Close": float(chunk["Adj Close"].iloc[0]),
        })
    return pd.DataFrame(rows) if rows else pd.DataFrame(columns=ohlc.columns.tolist())

In [4]:
def _compute_metrics(df: pd.DataFrame) -> tuple:
    """
    从 OHLC DataFrame 计算: 平均标准差(%), 平均High-Low Return(%), 平均TR%(%)。
    df 需含 Open, High, Low, Close, Adj Close，按时间降序。
    """
    if df is None or df.empty or len(df) < 2:
        return np.nan, np.nan, np.nan
    adj = df["Adj Close"]
    high = df["High"]
    low = df["Low"]
    open_ = df["Open"]
    close = df["Close"]
    if isinstance(adj, pd.DataFrame):
        adj, high, low, open_, close = adj.iloc[:, 0], high.iloc[:, 0], low.iloc[:, 0], open_.iloc[:, 0], close.iloc[:, 0]
    # C-C Returns
    cc_returns = (adj - adj.shift(-1)) / adj.shift(-1)
    std_pct = float(cc_returns.std() * 100) if cc_returns.notna().sum() >= 2 else np.nan
    # H-L Returns
    hl_returns = (high - low) / low
    mean_hl_pct = float(hl_returns.mean() * 100) if hl_returns.notna().any() else np.nan
    # TR%
    prev_close = close.shift(-1)
    tr = np.maximum(high - low, np.maximum(np.abs(high - prev_close), np.abs(low - prev_close)))
    tr_pct = tr / open_ * 100
    mean_tr_pct = float(tr_pct.mean()) if tr_pct.notna().any() else np.nan
    return std_pct, mean_hl_pct, mean_tr_pct


def volatility_metrics(
    symbol: str,
    start_date: str = "2010-01-01",
    end_date: str = None,
    provider: str = "yfinance",
) -> pd.DataFrame:
    """
    通过 OpenBB 获取标的历史数据，计算日、周、月、三月四个周期的：
    - 平均标准差（C-C 收益率标准差 %）
    - 平均 High-Low Return（%）
    - 平均 True Range Percentage（%）

    Returns:
        DataFrame: 单行结果，列包含 [symbol, Std_日(%), Std_周(%), ..., HL_日(%), ..., TR_日(%), ...]
        便于 pd.concat([volatility_metrics(s) for s in symbols], ignore_index=True) 批量对比
    """
    df_daily = _fetch_ohlc_openbb(symbol, start_date, end_date, provider)
    col_names = ["symbol", "Std_日(%)", "Std_周(%)", "Std_月(%)", "Std_三月(%)",
                 "HL_日(%)", "HL_周(%)", "HL_月(%)", "HL_三月(%)",
                 "TR_日(%)", "TR_周(%)", "TR_月(%)", "TR_三月(%)"]
    if df_daily is None or df_daily.empty:
        return pd.DataFrame(columns=col_names)

    row = {"symbol": symbol}
    # 日
    s, h, t = _compute_metrics(df_daily)
    row.update({"Std_日(%)": s, "HL_日(%)": h, "TR_日(%)": t})
    # 周、月、三月
    for label, n in [("周", 5), ("月", 20), ("三月", 60)]:
        resampled = _resample_ohlc(df_daily, n)
        s, h, t = _compute_metrics(resampled)
        row.update({f"Std_{label}(%)": s, f"HL_{label}(%)": h, f"TR_{label}(%)": t})

    out = pd.DataFrame([row])
    return out[col_names]  # 按指标分组: Std日周月季度 | HL日周月季度 | TR日周月季度

In [5]:
def get_atm_4w_iv(
    symbol: str,
    provider: str = "cboe",
    dte_range: tuple = (21, 35),
    option_type: str = "call",
) -> pd.DataFrame:
    """
    获取约 4 周后到期的 ATM 期权隐含波动率，并折算为日/周/月/三月期 IV。

    参数:
        symbol: 标的代码（如 SPY, AAPL）
        provider: 数据源，默认 cboe（美股期权）
        dte_range: (min_dte, max_dte) 天数范围，默认 21–35 天对应约 4 周
        option_type: 'call' 或 'put'，默认 call

    返回:
        DataFrame 单行: [标的代码, 日IV, 周IV, 月IV, 三月IV]，失败返回空 DataFrame
    """
    try:
        chain = obb.derivatives.options.chains(
            symbol=symbol,
            option_type=option_type,
            provider=provider,
        )
        col_names = ["标的代码", "日IV", "周IV", "月IV", "三月IV"]
        df = chain.to_df()
        if df is None or df.empty:
            return pd.DataFrame(columns=col_names)

        if "dte" not in df.columns or "implied_volatility" not in df.columns:
            return pd.DataFrame(columns=col_names)

        df = df[(df["dte"] >= dte_range[0]) & (df["dte"] <= dte_range[1])]
        if df.empty:
            return pd.DataFrame(columns=col_names)

        spot = df["underlying_price"].iloc[0]
        if pd.isna(spot) or spot <= 0:
            return pd.DataFrame(columns=col_names)

        df = df.copy()
        df["_atm_dist"] = abs(df["strike"] - spot) / spot
        best = df.loc[df["_atm_dist"].idxmin()]
        iv = best["implied_volatility"]
        if pd.isna(iv):
            return pd.DataFrame(columns=col_names)

        # 年化 IV 折算为各周期 IV（小数形式）
        three_month_iv = iv * np.sqrt(1 / 4)
        monthly_iv = iv * np.sqrt(1 / 12)
        weekly_iv = iv * np.sqrt(1 / 52)
        daily_iv = iv * np.sqrt(1 / 365)

        return pd.DataFrame([{
            "标的代码": symbol,
            "日IV": daily_iv,
            "周IV": weekly_iv,
            "月IV": monthly_iv,
            "三月IV": three_month_iv,
        }])
    except Exception:
        return pd.DataFrame(columns=["标的代码", "日IV", "周IV", "月IV", "三月IV"])

In [13]:
# 示例：获取 SPY 约 4 周后到期 ATM call 的 IV，并按周期折算
result = get_atm_4w_iv("SPY")
display(result)

,标的代码,日IV,周IV,月IV,三月IV
0,SPY,0.009898,0.026223,0.054588,0.09455


In [7]:
# Asset universe: Asset, Class, Notes
assets_df = pd.DataFrame([
    # Government Bonds
    ("TLT", "Government Bonds", "20+ yr"),
    ("IEF", "Government Bonds", "7-10 yr"),
    ("IEI", "Government Bonds", "3-7 yr"),
    ("SHY", "Government Bonds", "1-3 yr"),
    # Corporate Bonds
    ("LQD", "Corporate Bonds", "Investment Grade"),
    ("VCIT", "Corporate Bonds", "Investment Grade"),
    ("JNK", "Corporate Bonds", "Junk"),
    ("HYG", "Corporate Bonds", "Junk"),
    # FX Major
    ("EURUSD", "FX Major", None),
    ("USDJPY", "FX Major", None),
    ("GBPUSD", "FX Major", None),
    ("AUDUSD", "FX Major", None),
    # FX EM
    ("USDZAR", "FX EM", None),
    ("USDBRL", "FX EM", None),
    ("USDTRY", "FX EM", None),
    # Equity Index
    ("SPY", "Equity Index", "S&P 500"),
    ("QQQ", "Equity Index", "Nasdaq 100"),
    ("DIA", "Equity Index", "Dow Jones Industrial"),
    ("IWV", "Equity Index", "Russell 3000"),
    ("IWM", "Equity Index", "Russell 2000"),
    # Equity - Mega Cap (FAANMG)
    ("META", "Equity - Mega Cap", "FAANMG"),
    ("AAPL", "Equity - Mega Cap", "FAANMG"),
    ("AMZN", "Equity - Mega Cap", "FAANMG"),
    ("NFLX", "Equity - Mega Cap", "FAANMG"),
    ("MSFT", "Equity - Mega Cap", "FAANMG"),
    ("GOOG", "Equity - Mega Cap", "FAANMG"),
    # Equity - Large Cap
    ("MAR", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LVS", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LEN", "Equity - Large Cap", "Discretionary/Industrial"),
    ("BBY", "Equity - Large Cap", "Discretionary/Industrial"),
    ("ODFL", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LUV", "Equity - Large Cap", "Discretionary/Industrial"),
    ("PCAR", "Equity - Large Cap", "Discretionary/Industrial"),
    ("JCI", "Equity - Large Cap", "Discretionary/Industrial"),
    ("WMT", "Equity - Large Cap", "Discretionary/Industrial/Tech?"),
    ("HSY", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("CPB", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("STZ", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("MNST", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("BIIB", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("ALGN", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("XEL", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("PPL", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("SNDK", "Equity - Large Cap", "Semiconductor"),
    ("MU", "Equity - Large Cap", "Semiconductor"),
    ("TSM", "Equity - Large Cap", "Semiconductor"),
    ("PLTR", "Equity - Large Cap", "AI/Military"),
    # Equity - Mid Cap
    ("WING", "Equity - Mid Cap", None),
    ("FRPT", "Equity - Mid Cap", None),
    ("BILL", "Equity - Mid Cap", None),
    ("PEGA", "Equity - Mid Cap", None),
    ("OLED", "Equity - Mid Cap", None),
    ("XPO", "Equity - Mid Cap", None),
    ("FND", "Equity - Mid Cap", None),
    ("TREX", "Equity - Mid Cap", None),
    ("MOS", "Equity - Mid Cap", None),
    ("OLLI", "Equity - Mid Cap", None),
    ("DKS", "Equity - Mid Cap", None),
    ("VIRT", "Equity - Mid Cap", None),
    ("JBLU", "Equity - Mid Cap", None),
], columns=["Asset", "Class", "Notes"])

In [8]:
FX_TICKERS = {"EURUSD", "USDJPY", "GBPUSD", "AUDUSD", "USDZAR", "USDBRL", "USDTRY"}


def _to_openbb_symbol(asset: str) -> str:
    """FX 标的需加 =X 后缀才能被 yfinance/OpenBB 识别"""
    return f"{asset}=X" if asset in FX_TICKERS else asset


# 循环计算所有标的的波动率指标，拼接至 assets_df（网络异常自动重试）
metric_cols = [
    "Std_日(%)", "Std_周(%)", "Std_月(%)", "Std_三月(%)",
    "HL_日(%)", "HL_周(%)", "HL_月(%)", "HL_三月(%)",
    "TR_日(%)", "TR_周(%)", "TR_月(%)", "TR_三月(%)",
]
RETRY_MAX, RETRY_DELAY = 3, 5
metrics_list = []
for asset in assets_df["Asset"]:
    sym = _to_openbb_symbol(asset)
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            row = volatility_metrics(sym).iloc[0].to_dict()
            row["Asset"] = asset
            metrics_list.append(row)
            break
        except Exception as e:
            last_err = e
            if attempt < RETRY_MAX - 1:
                time.sleep(RETRY_DELAY)
            else:
                metrics_list.append({"Asset": asset, **{c: np.nan for c in metric_cols}})
                print(f"{asset} 波动率失败(重试{RETRY_MAX}次后): {type(e).__name__}")

metrics_df = pd.DataFrame(metrics_list).drop(columns=["symbol"], errors="ignore")
assets_df = assets_df.merge(metrics_df, on="Asset", how="left")
# 列顺序: Asset, Class, Notes | Std日周月季度 | HL日周月季度 | TR日周月季度
assets_df = assets_df[["Asset", "Class", "Notes"] + [c for c in metric_cols if c in assets_df.columns]]

# 循环计算每个标的的日周月季度 IV（期权链，债券/FX 等无期权则填 NaN，网络异常自动重试）
iv_list = []
iv_cols = ["日IV(%)", "周IV(%)", "月IV(%)", "三月IV(%)"]
for asset in assets_df["Asset"]:
    df_iv = None
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            df_iv = get_atm_4w_iv(asset)
            break
        except Exception as e:
            last_err = e
            if attempt < RETRY_MAX - 1:
                time.sleep(RETRY_DELAY)
    if df_iv is not None and not df_iv.empty:
        r = df_iv.iloc[0]
        iv_list.append({
            "Asset": asset,
            "日IV(%)": r["日IV"] * 100 if pd.notna(r["日IV"]) else np.nan,
            "周IV(%)": r["周IV"] * 100 if pd.notna(r["周IV"]) else np.nan,
            "月IV(%)": r["月IV"] * 100 if pd.notna(r["月IV"]) else np.nan,
            "三月IV(%)": r["三月IV"] * 100 if pd.notna(r["三月IV"]) else np.nan,
        })
    else:
        iv_list.append({"Asset": asset, "日IV(%)": np.nan, "周IV(%)": np.nan, "月IV(%)": np.nan, "三月IV(%)": np.nan})
        if last_err is not None:
            print(f"{asset} 期权IV失败(重试{RETRY_MAX}次后): {type(last_err).__name__}")

iv_df = pd.DataFrame(iv_list)
assets_df = assets_df.merge(iv_df, on="Asset", how="left")
final_cols = ["Asset", "Class", "Notes"] + [c for c in metric_cols + iv_cols if c in assets_df.columns]
assets_df = assets_df[final_cols]


1 Failed download:
['SNDK']: Timeout('Failed to perform, curl: (28) Connection timed out after 10000 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


In [9]:
assets_df.to_csv("export/openBB_volatility_metrics.csv", index=False)